In [ ]:
!pip install --quiet langchain_google_genai langchain_community langgraph langgraph-prebuilt pyowm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 89.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 13.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of

In [ ]:
import os
import getpass


GEMINI_API_KEY = getpass.getpass("Enter Gemini API Key: ")
os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY

from langchain_community.utilities import OpenWeatherMapAPIWrapper

OPENWEATHERMAP_API_KEY = getpass.getpass("Enter OpenWeatherMap API Key: ")
os.environ["OPENWEATHERMAP_API_KEY"]=OPENWEATHERMAP_API_KEY


Enter Gemini API Key: ··········
Enter OpenWeatherMap API Key: ··········


In [ ]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.tools.openweathermap import OpenWeatherMapQueryRun
from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, END, MessagesState

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

tools = OpenWeatherMapQueryRun(api_key=os.environ["OPENWEATHERMAP_API_KEY"])

model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)
model_with_tools = model.bind_tools([tools])

tool_node = ToolNode([tools])

def process_user_input(state):
    messages = state['messages']
    response = model_with_tools.invoke(messages)
    return {"messages": [response]}


def should_continue(state):
    # Determine whether to continue with tool execution or end.
    messages = state['messages']
    last_message = messages[-1]

    # If the LLM makes a tool call, then we route to the "tools" node
    if last_message.tool_calls:
        return "tools"

    return END

# Create the workflow using StateGraph
workflow = StateGraph(AgentState)

# Add nodes to the workflow
workflow.add_node("agent", process_user_input)
workflow.add_node("tools", tool_node)

# Set the entry point
workflow.set_entry_point("agent")

# Add conditional edges
workflow.add_conditional_edges( "agent",
                                should_continue,
                                {
                                  "tools": "tools",
                                  END: END
                                }
                              )

# Add edge from tools back to agent
workflow.add_edge('tools', "agent")

# Compile the workflow
app = workflow.compile()

inputs = {"messages": [HumanMessage(content="Hey! how are you?")]}
result = app.invoke(inputs)
for m in result['messages']:
  m.pretty_print()


================================ Human Message =================================

Hey! how are you?
================================== Ai Message ==================================

I'm doing well, thank you for asking! How can I help you today?


In [ ]:
inputs = {"messages": [HumanMessage(content="What is temperatue in Atlanta")]}
for output in app.stream(inputs):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'agent':
---
{'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'open_weather_map', 'arguments': '{"location": "Atlanta,US"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run--1a10646d-54f1-49aa-bb08-f29ac876e0c9-0', tool_calls=[{'name': 'open_weather_map', 'args': {'location': 'Atlanta,US'}, 'id': '6ff5087d-6536-4560-b784-bc0b32d47c3e', 'type': 'tool_call'}], usage_metadata={'input_tokens': 49, 'output_tokens': 9, 'total_tokens': 58, 'input_token_details': {'cache_read': 0}})]}

---

Output from node 'tools':
---
{'messages': [ToolMessage(content='In Atlanta,US, the current weather is as follows:\nDetailed status: overcast clouds\nWind speed: 2.57 m/s, direction: 260°\nHumidity: 66%\nTemperature: \n  - Current: 26.8°C\n  - High: 27.72°C\n  - Low: 25.09°C\n  - Feels like: 28.24°C\nRain: {}\nHeat index: None\nCloud co